In [27]:
!pip install wget 

In [28]:
!pip install scikit-learn 

In [29]:
!pip install "ibm-watson-machine-learning>=1.0.310" 

In [30]:
credentials = {
    "url":"https://eu-de.ml.cloud.ibm.com" ,
    "apikey":"xicBYxJT6RfCUvO35Kvg21OQOUEk32FG0qwcZkUgJtsa"
}

In [31]:
import os

In [32]:
try:
    project_id=os.environ["PROJECT_ID"]

except KeyError:
    project_id = input("please enter your project_id:")

In [33]:
project_id

'b5058e8b-3d98-4e45-9b71-e39b4ae5f982'

In [34]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='IcXY9sbCKgaK7ZpjkV7MdDtT9941lWVTf0ejXeyx44mc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'lashassandbox-donotdelete-pr-7asd0ktkznumzv'
object_key = 'legal_sentiment_dataset.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head(10)

,ID,Phrase,Sentiment
0,1,The plaintiff's claims are dismissed.,-1
1,2,The contract is deemed valid and enforceable.,1
2,3,The appeal is denied due to lack of merit.,-1
3,4,The legal team submitted additional evidence.,0
4,5,The appeal is under consideration.,0
5,6,The settlement agreement is fair and reasonable.,1
6,7,The appeal is under consideration.,0
7,8,The settlement agreement is fair and reasonable.,1
8,9,The court reviewed the documentation.,0
9,10,The contract is deemed valid and enforceable.,1


In [35]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='IcXY9sbCKgaK7ZpjkV7MdDtT9941lWVTf0ejXeyx44mc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'lashassandbox-donotdelete-pr-7asd0ktkznumzv'
object_key = 'legal_sentiment_dataset.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head(10)

,ID,Phrase,Sentiment
0,1,The plaintiff's claims are dismissed.,-1
1,2,The contract is deemed valid and enforceable.,1
2,3,The appeal is denied due to lack of merit.,-1
3,4,The legal team submitted additional evidence.,0
4,5,The appeal is under consideration.,0
5,6,The settlement agreement is fair and reasonable.,1
6,7,The appeal is under consideration.,0
7,8,The settlement agreement is fair and reasonable.,1
8,9,The court reviewed the documentation.,0
9,10,The contract is deemed valid and enforceable.,1


In [36]:
label_map={
    -1:"negative",
    0:"neural",
    1:"positive"
}

In [37]:
data.value_counts(['Sentiment'])

Sentiment
 1           184
-1           167
 0           149
Name: count, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split

data_train,data_test,y_train,y_test=train_test_split(data['Phrase'],
                                                     data['Sentiment'],
                                                     test_size=0.3,
                                                     random_state=33,
                                                     stratify=data['Sentiment'])
print(type(data_train))
data_train=pd.DataFrame(data_train)
data_test=pd.DataFrame(data_test)

<class 'pandas.core.series.Series'>


In [39]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL']


In [40]:
model_id=ModelTypes.FLAN_T5_XXL

In [41]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [42]:
parameters={
    GenParams.DECODING_METHOD:"Greedy",
    GenParams.RANDOM_SEED:33,
    GenParams.REPETITION_PENALTY:1,
    GenParams.MIN_NEW_TOKENS:1,
    GenParams.MAX_NEW_TOKENS:1,
    
}

In [43]:
from ibm_watson_machine_learning.foundation_models import Model

In [44]:
model=Model(

    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

In [45]:
 model.get_details()

{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'terms_url': 'https://huggingface.co/google/flan-t5-xxl/blob/main/README.md',
 'input_tier': 'class_2',
 'output_tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'

In [46]:
!pip install transformers

In [47]:
from transformers import pipeline
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Assuming data_train, data_test, y_train, y_test are already defined from your notebook 
# If not, include the data loading and splitting code from the original notebook

# Load a pre-trained sentiment analysis model that supports multiple classes
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment", device=-1) # device=-1 for CPU

# Predict sentiments for the test set
predictions = classifier(data_test['Phrase'].tolist())

# Map the model's outputs to your labels
# The model returns star ratings (1 to 5), which we map to -1 (negative), 0 (neutral), 1 (positive)
def map_to_sentiment(label):
    if label in ['1 star', '2 stars']:
        return -1 # Negative
    elif label == '3 stars':
        return 0 # Neutral
    elif label in ['4 stars', '5 stars']:
        return 1 # Positive
    return 0 # Default to neutral for unexpected outputs
    
numeric_predictions = [map_to_sentiment(pred['label']) for pred in predictions]

# Evaluate model performance
accuracy = accuracy_score(y_test, numeric_predictions) 
print(f"Accuracy: {accuracy:.4f}")

# Generate classification report with correct target_names
print("\nClassification Report:")
print(classification_report(y_test, numeric_predictions, target_names=["negative", "neutral", "positive"]))

# Display sample predictions
sample_results = pd.DataFrame({
'Phrase': data_test['Phrase'],
'Actual_Sentiment': y_test.map({-1: "negative", 0: "neutral", 1: "positive"}), 
'Predicted_Sentiment': [label_map[pred] for pred in numeric_predictions]
})
print("\nSample Predictions:") 
print(sample_results.head(10))

Device set to use cpu


Accuracy: 0.6533

Classification Report:
              precision    recall  f1-score   support

    negative       0.52      1.00      0.68        50
     neutral       1.00      0.42      0.59        45
    positive       0.83      0.53      0.64        55

    accuracy                           0.65       150
   macro avg       0.78      0.65      0.64       150
weighted avg       0.78      0.65      0.64       150


Sample Predictions:
                                                Phrase Actual_Sentiment  \
489      The legal team submitted additional evidence.          neutral   
459         The appeal is denied due to lack of merit.         negative   
401           The hearing is scheduled for next month.          neutral   
458  The case was transferred to another jurisdiction.          neutral   
156                  The motion to dismiss is granted.         positive   
409           The hearing is scheduled for next month.          neutral   
7     The settlement agreement i